In [16]:
# This is to create the base model 

In [17]:
import os
os.chdir("../")

In [18]:
from deepClassifier.constants.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from deepClassifier.utils.common import read_yaml, create_directories

In [19]:
PrepareBaseModel = namedtuple("PrepareBaseModel", [
    "root_dir",
    "base_model_path",
    "updated_base_model_path"
    ])

NameError: name 'namedtuple' is not defined

In [20]:
from dataclasses import dataclass
from importlib.resources import path
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: str
    base_model_path: Path
    updated_base_model_path : Path
    IMAGE_SIZE: list
    BATCH_SIZE: int
    INCLUDE_TOP: bool
    EPOCHS: int
    CLASSES: int
    WEIGHTS: str
    LEARNING_RATE: float
 

In [21]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_Base_Model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        PrepareBaseModelConfig = PrepareBaseModel(

            root_dir = config.root_dir,
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = path(config.updated_base_model_path),
            IMAGE_SIZE = config.IMAGE_SIZE,
            BATCH_SIZE = config.BATCH_SIZE,
            INCLUDE_TOP = config.INCLUDE_TOP,
            EPOCHS = config.EPOCHS,
            CLASSES = config.CLASSES,
            WEIGHTS = config.WEIGHTS,
            LEARNING_RATE = config.LEARNING_RATE
            )

        
        
        
        
        

        return PrepareBaseModelConfig


In [22]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        

In [23]:
try:
    config = ConfigurationManager()
    Preparebasemodel_config = config.get_Base_Model_config()
    Preparebasemodel = PrepareBaseModel(config=Preparebasemodel_config)
    Preparebasemodel.get_base_model()
    Preparebasemodel.updated_base_model()
    Preparebasemodel.save_base_model()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'configs\\config.yaml'